In [2]:
!pip install -q pyyaml h5py

In [3]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28)/255.0
test_images = test_images[:1000].reshape(-1, 28*28)/255.0

print(test_images)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])
#인풋 레이어를 지정하고 그 위에 레이어들은 인풋을 설정안해도된다.
#Dropout : 인풋유닛을 0으로 설정하는 비율(과적합 방지)
  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
#os.path.dirname : 경로 중 디렉토리명만 얻기 ( training_1)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)
# filepath : 모델을 저장할 경로, save_weights_only : True일 경우, 나오는 값 중에 가장 좋은 값이 모델에 저장, verbose=1 : 저장되었다고 화면에 표시

In [7]:
keras.callbacks.Callback()

!pip install callbacks

In [8]:
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])
# fit 안에 위의 콜백 함수를 넣어야한다.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
  32/1000 [..............................] - ETA: 11s - loss: 2.3434 - accuracy: 0.0938
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 532us/sample - loss: 1.1284 - accuracy: 0.6840 - val_loss: 0.6965 - val_accuracy: 0.7940
Epoch 2/10
  32/1000 [..............................] - ETA: 0s - loss: 0.6362 - accuracy: 0.8125
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 103us/sample - loss: 0.4136 - accuracy: 0.8780 - val_loss: 0.5256 - val_accuracy: 0.8360
Epoch 3/10
  32/1000 [..............................] - ETA: 0s - loss: 0.4011 - accuracy: 0.8438
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 107us/sample - loss: 0.2972 - accuracy: 0.9200 - val_loss: 0.4970 - val_accuracy: 0.8370
Epoch 4/10
  32/1000 [..............................] - ETA: 0s - loss: 0.0771 - accuracy: 1.0000
Epoc

In [9]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00002  cp.ckpt.index
cp.ckpt/    cp.ckpt.data-00001-of-00002


In [10]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도 : {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.4823 - accuracy: 0.1090
훈련되지 않은 모델의 정확도 : 10.90%


In [11]:
model.load_weights(checkpoint_path)

In [12]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도 : {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.3994 - accuracy: 0.8690
복원된 모델의 정확도 : 86.90%


In [13]:
checkpoin_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#에포크 5일 때 마다 가중치를 저장

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weight_only=True,
    period=5)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))


#위에서 콜백을 사용하기때문에 훈련때도 사용
model.fit(train_images,
         train_labels,
         epochs=50,
         callbacks=[cp_callback],
         validation_data=(test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_1/cp.ckpt
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets

Epoch 00010: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets

Epoch 00015: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets

Epoch 00020: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets

Epoch 00025: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets

Epoch 00030: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets

Epoch 00035: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets

Epoch 00040: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets

Epoch 00045: saving model to training_

In [14]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00002  cp.ckpt.index
cp.ckpt/    cp.ckpt.data-00001-of-00002


In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_1/cp.ckpt'

In [16]:
# 모델을 초기화 하고 새로운 객체 생성

model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도 : {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.4689 - accuracy: 0.1020
복원된 모델의 정확도 : 10.20%


In [17]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose = 2)

1000/1 - 0s - loss: 2.4689 - accuracy: 0.1020


In [18]:
# 전체 모델 저장하기

model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 217us/sample - loss: 1.1406 - accuracy: 0.6860
Epoch 2/5
1000/1000 [==============================] - 0s 45us/sample - loss: 0.3967 - accuracy: 0.8920
Epoch 3/5
1000/1000 [==============================] - 0s 47us/sample - loss: 0.2800 - accuracy: 0.9280
Epoch 4/5
1000/1000 [==============================] - 0s 47us/sample - loss: 0.2124 - accuracy: 0.9460
Epoch 5/5
1000/1000 [==============================] - 0s 45us/sample - loss: 0.1704 - accuracy: 0.9620
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [22]:
#왜 오류나지..?

ls saved_model

ls saved_model/my_model

SyntaxError: invalid syntax (<ipython-input-22-ee2eacd4e5c2>, line 1)

In [23]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도 : {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape) # 행, 열 표시

1000/1 - 0s - loss: 0.4824 - accuracy: 0.8600
복원된 모델의 정확도 : 86.00%
(1000, 10)


In [26]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 216us/sample - loss: 1.1488 - accuracy: 0.6790
Epoch 2/5
1000/1000 [==============================] - 0s 45us/sample - loss: 0.4318 - accuracy: 0.8710
Epoch 3/5
1000/1000 [==============================] - 0s 44us/sample - loss: 0.2959 - accuracy: 0.9240
Epoch 4/5
1000/1000 [==============================] - 0s 46us/sample - loss: 0.2088 - accuracy: 0.9510
Epoch 5/5
1000/1000 [==============================] - 0s 47us/sample - loss: 0.1579 - accuracy: 0.9670


In [27]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [28]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)

print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

1000/1 - 0s - loss: 0.6069 - accuracy: 0.8630
복원된 모델의 정확도: 86.30%


In [29]:
# 가중치 값, 모델 구조, 모델의 훈련설정(컴파일), 옵티마이저를 저장